In [2]:
import geopandas as gpd

gdb_path = "../data/raw/ICADisplay.gdb"
feeder = gpd.read_file(gdb_path, driver='fileGDB', layer="FeederDetail")
# feeder_load = gpd.read_file(gdb_path, driver='fileGDB', layer="FeederLoadProfile")
line = gpd.read_file(gdb_path, driver='fileGDB', layer="LineDetail")
substations = gpd.read_file(gdb_path, driver='fileGDB', layer="Substations")
# substation_load = gpd.read_file(gdb_path, driver='fileGDB', layer="SubstationLoadProfile")

In [3]:
# Convert to GeoJSON and save to file
feeder = feeder.to_crs(epsg=4326)
feeder.to_file("../data/geojson/feeder.geojson", driver='GeoJSON')

In [4]:
# Feeder less than 100 kW
import pandas as pd

aggregated_capacity = line.groupby('FeederId').agg({'LoadCapacity_kW': 'sum'}).reset_index()
feeder_capacity = pd.merge(feeder, aggregated_capacity, left_on='FeederID', right_on='FeederId', how='inner')

capacity = 100 # 100 kW = low
low_capacity_feeders = feeder_capacity[feeder_capacity['LoadCapacity_kW'] < capacity]

low_capacity_lines = pd.merge(line, low_capacity_feeders[['FeederID']], left_on='FeederId', right_on='FeederID', how='inner')
low_capacity_lines_gdf = gpd.GeoDataFrame(low_capacity_lines, geometry='geometry')

low_capacity_lines_gdf.to_file("../data/geojson/low_capacity_feeders.geojson", driver='GeoJSON')

In [5]:
line = line.to_crs(epsg=4326)
line.to_file("../data/geojson/line.geojson", driver='GeoJSON')

In [6]:
substations = substations.to_crs(epsg=4326)
substations.to_file("../data/geojson/substations.geojson", driver='GeoJSON')
# substation_load.to_file("../data/geojson/substation_load.geojson", driver='GeoJSON')

In [7]:
import geopandas as gpd
import pandas as pd

# Paths to the data
gdb_path = "../data/raw/ICADisplay.gdb"

# Reading the data
feeder = gpd.read_file(gdb_path, driver='fileGDB', layer="FeederDetail")
line = gpd.read_file(gdb_path, driver='fileGDB', layer="LineDetail")

# Converting to the correct coordinate system
feeder = feeder.to_crs(epsg=4326)
line = line.to_crs(epsg=4326)

In [8]:
# # Aggregating load capacity by FeederId
# # aggregated_capacity = line.groupby('FeederId').agg({'LoadCapacity_kW': 'sum'}).reset_index()

# min_line_seg_capacity = line.groupby('FeederId').agg({'LoadCapacity_kW': 'min'}).reset_index()

# min_line_seg_capacity.head()

# Aggregating load capacity by FeederId
quantile_line_segment_capacity = line.groupby('FeederId').agg({'LoadCapacity_kW': lambda x: x.quantile(0.8)}).reset_index()

# Merging feeder data with the line segment capacity in the selected feeder
feeder_capacity = pd.merge(feeder, quantile_line_segment_capacity, left_on='FeederID', right_on='FeederId', how='inner')

In [60]:
# Merging feeder data with aggregated capacity
# feeder_capacity = pd.merge(feeder, aggregated_capacity, left_on='FeederID', right_on='FeederId', how='inner')

# Check columns after merge
print("Feeder Capacity Columns:", feeder_capacity.columns)

# Filtering for MinElectrification
min_electrification = feeder_capacity[feeder_capacity['ResCust'] * 18 / 2 <= feeder_capacity['LoadCapacity_kW']]

mid_electrification = feeder_capacity[feeder_capacity['ResCust'] * 20.4 / 2 <= feeder_capacity['LoadCapacity_kW']]

max_electrification = feeder_capacity[feeder_capacity['ResCust'] * 31.9 / 2 <= feeder_capacity['LoadCapacity_kW']]

Feeder Capacity Columns: Index(['Substation', 'FeederID', 'Feeder_Name', 'Nominal_Voltage',
       'Load_Profile_Redaction', 'ResCust', 'ComCust', 'IndCust', 'AgrCust',
       'OthCust', 'Existing_DG', 'Queued_DG', 'Total_DG', 'Show', 'VOLTNUM',
       'Shape_Length', 'geometry', 'FeederId', 'LoadCapacity_kW'],
      dtype='object')


In [61]:
# min_electrification.count()
mid_electrification.count()

Substation                344
FeederID                  344
Feeder_Name               344
Nominal_Voltage           344
Load_Profile_Redaction    344
ResCust                   344
ComCust                   344
IndCust                   344
AgrCust                   344
OthCust                   344
Existing_DG               344
Queued_DG                 344
Total_DG                  344
Show                      344
VOLTNUM                   344
Shape_Length              344
geometry                  344
FeederId                  344
LoadCapacity_kW           344
dtype: int64

In [62]:
mid_electrification.head()

,Substation,FeederID,Feeder_Name,Nominal_Voltage,Load_Profile_Redaction,ResCust,ComCust,IndCust,AgrCust,OthCust,Existing_DG,Queued_DG,Total_DG,Show,VOLTNUM,Shape_Length,geometry,FeederId,LoadCapacity_kW
3,BUENA VISTA,182261106,BUENA VISTA 1106,12kV,Yes,14,25,18,49,1,200,200,400,1,12.0,23941.399005,"MULTILINESTRING ((-121.61371 36.64274, -121.61...",182261106,3750.0
6,BARTON,253571101,BARTON 1101,12kV,Yes,83,313,112,2,6,600,240,840,1,12.0,21732.210875,"MULTILINESTRING ((-119.69086 36.77227, -119.69...",253571101,1550.0
15,SAN JOSE A,082251113,SAN JOSE A 1113,12kV,Yes,0,13,20,0,3,390,0,390,1,12.0,2881.396061,"MULTILINESTRING ((-121.89326 37.33215, -121.89...",082251113,370.0
20,SAN RAFAEL,042011103,SAN RAFAEL 1103,12kV,Yes,49,635,128,0,14,1430,3210,4640,1,12.0,21771.435505,"MULTILINESTRING ((-122.48469 37.94618, -122.48...",042011103,1172.0
27,MT EDEN,013761114,MT EDEN 1114,12kV,Yes,4,249,125,0,5,560,2790,3350,1,12.0,20658.584954,"MULTILINESTRING ((-122.12877 37.62509, -122.12...",013761114,3448.0


In [63]:
max_electrification.head()

,Substation,FeederID,Feeder_Name,Nominal_Voltage,Load_Profile_Redaction,ResCust,ComCust,IndCust,AgrCust,OthCust,Existing_DG,Queued_DG,Total_DG,Show,VOLTNUM,Shape_Length,geometry,FeederId,LoadCapacity_kW
3,BUENA VISTA,182261106,BUENA VISTA 1106,12kV,Yes,14,25,18,49,1,200,200,400,1,12.0,23941.399005,"MULTILINESTRING ((-121.61371 36.64274, -121.61...",182261106,3750.0
6,BARTON,253571101,BARTON 1101,12kV,Yes,83,313,112,2,6,600,240,840,1,12.0,21732.210875,"MULTILINESTRING ((-119.69086 36.77227, -119.69...",253571101,1550.0
15,SAN JOSE A,082251113,SAN JOSE A 1113,12kV,Yes,0,13,20,0,3,390,0,390,1,12.0,2881.396061,"MULTILINESTRING ((-121.89326 37.33215, -121.89...",082251113,370.0
20,SAN RAFAEL,042011103,SAN RAFAEL 1103,12kV,Yes,49,635,128,0,14,1430,3210,4640,1,12.0,21771.435505,"MULTILINESTRING ((-122.48469 37.94618, -122.48...",042011103,1172.0
27,MT EDEN,013761114,MT EDEN 1114,12kV,Yes,4,249,125,0,5,560,2790,3350,1,12.0,20658.584954,"MULTILINESTRING ((-122.12877 37.62509, -122.12...",013761114,3448.0


In [64]:
# Function to create GeoDataFrame and save to GeoJSON
def create_geojson(df, line, filename):
    filtered_lines = pd.merge(line, df[['FeederID']], left_on='FeederId', right_on='FeederID', how='inner')
    print(f"Columns after first merge for {filename}:", filtered_lines.columns)
    filtered_lines = pd.merge(filtered_lines, df[['FeederID', 'Feeder_Name', 'LoadCapacity_kW', 'ResCust', 'ComCust', 'IndCust']], on='FeederID', how='left')  # Merge additional metadata
    print(f"Columns after second merge for {filename}:", filtered_lines.columns)
    filtered_lines_gdf = gpd.GeoDataFrame(filtered_lines, geometry='geometry')
    filtered_lines_gdf = filtered_lines_gdf.rename(columns={
        'LoadCapacity_kW_y': 'LoadCapacity_kW'
    })
    filtered_lines_gdf = filtered_lines_gdf[[
        'Feeder_Name', 'FeederID', 'LoadCapacity_kW', 'ResCust', 'ComCust', 'IndCust', 'geometry'
    ]]
    filtered_lines_gdf.to_crs(epsg=4326).to_file(f"../data/geojson/{filename}.geojson", driver='GeoJSON')

# Create GeoJSON files for each electrification level
create_geojson(min_electrification, line, "min_electrification_halfcust")
create_geojson(mid_electrification, line, "mid_electrification_halfcust")
create_geojson(max_electrification, line, "max_electrification_halfcust")

Columns after first merge for min_electrification_halfcust: Index(['FeederId', 'FeederName', 'Globalid', 'CSV_LineSection',
       'ICA_Analysis_Date', 'LoadCapacity_kW', 'GenCapacity_kW',
       'GenericPVCapacity_kW', 'GenCapacity_no_OpFlex_kW',
       'GenericCapacity_no_OpFlex_kW', 'Shape_Length', 'geometry', 'FeederID'],
      dtype='object')
Columns after second merge for min_electrification_halfcust: Index(['FeederId', 'FeederName', 'Globalid', 'CSV_LineSection',
       'ICA_Analysis_Date', 'LoadCapacity_kW_x', 'GenCapacity_kW',
       'GenericPVCapacity_kW', 'GenCapacity_no_OpFlex_kW',
       'GenericCapacity_no_OpFlex_kW', 'Shape_Length', 'geometry', 'FeederID',
       'Feeder_Name', 'LoadCapacity_kW_y', 'ResCust', 'ComCust', 'IndCust'],
      dtype='object')
Columns after first merge for mid_electrification_halfcust: Index(['FeederId', 'FeederName', 'Globalid', 'CSV_LineSection',
       'ICA_Analysis_Date', 'LoadCapacity_kW', 'GenCapacity_kW',
       'GenericPVCapacity_kW', '

In [65]:
create_geojson(min_electrification, line, "min_electrification")

Columns after first merge for min_electrification: Index(['FeederId', 'FeederName', 'Globalid', 'CSV_LineSection',
       'ICA_Analysis_Date', 'LoadCapacity_kW', 'GenCapacity_kW',
       'GenericPVCapacity_kW', 'GenCapacity_no_OpFlex_kW',
       'GenericCapacity_no_OpFlex_kW', 'Shape_Length', 'geometry', 'FeederID'],
      dtype='object')
Columns after second merge for min_electrification: Index(['FeederId', 'FeederName', 'Globalid', 'CSV_LineSection',
       'ICA_Analysis_Date', 'LoadCapacity_kW_x', 'GenCapacity_kW',
       'GenericPVCapacity_kW', 'GenCapacity_no_OpFlex_kW',
       'GenericCapacity_no_OpFlex_kW', 'Shape_Length', 'geometry', 'FeederID',
       'Feeder_Name', 'LoadCapacity_kW_y', 'ResCust', 'ComCust', 'IndCust'],
      dtype='object')


In [66]:
create_geojson(mid_electrification, line, "mid_electrification")


Columns after first merge for mid_electrification: Index(['FeederId', 'FeederName', 'Globalid', 'CSV_LineSection',
       'ICA_Analysis_Date', 'LoadCapacity_kW', 'GenCapacity_kW',
       'GenericPVCapacity_kW', 'GenCapacity_no_OpFlex_kW',
       'GenericCapacity_no_OpFlex_kW', 'Shape_Length', 'geometry', 'FeederID'],
      dtype='object')
Columns after second merge for mid_electrification: Index(['FeederId', 'FeederName', 'Globalid', 'CSV_LineSection',
       'ICA_Analysis_Date', 'LoadCapacity_kW_x', 'GenCapacity_kW',
       'GenericPVCapacity_kW', 'GenCapacity_no_OpFlex_kW',
       'GenericCapacity_no_OpFlex_kW', 'Shape_Length', 'geometry', 'FeederID',
       'Feeder_Name', 'LoadCapacity_kW_y', 'ResCust', 'ComCust', 'IndCust'],
      dtype='object')


In [67]:
create_geojson(max_electrification, line, "max_electrification")

Columns after first merge for max_electrification: Index(['FeederId', 'FeederName', 'Globalid', 'CSV_LineSection',
       'ICA_Analysis_Date', 'LoadCapacity_kW', 'GenCapacity_kW',
       'GenericPVCapacity_kW', 'GenCapacity_no_OpFlex_kW',
       'GenericCapacity_no_OpFlex_kW', 'Shape_Length', 'geometry', 'FeederID'],
      dtype='object')
Columns after second merge for max_electrification: Index(['FeederId', 'FeederName', 'Globalid', 'CSV_LineSection',
       'ICA_Analysis_Date', 'LoadCapacity_kW_x', 'GenCapacity_kW',
       'GenericPVCapacity_kW', 'GenCapacity_no_OpFlex_kW',
       'GenericCapacity_no_OpFlex_kW', 'Shape_Length', 'geometry', 'FeederID',
       'Feeder_Name', 'LoadCapacity_kW_y', 'ResCust', 'ComCust', 'IndCust'],
      dtype='object')
